1. Ładowanie modelu klasyfikatora

In [12]:
from tensorflow.keras.models import load_model

# Ścieżka do modelu klasyfikatora
model_path = "Classifier\PatoGenClassifier.keras"
classifier_model = load_model(model_path)
print("Model klasyfikacji załadowany pomyślnie!")

Model klasyfikacji załadowany pomyślnie!


2. Ładowanie modelu YOLO i detekcja obszarów

In [13]:
from ultralyticsplus import YOLO, render_result
from PIL import Image

# Ścieżka do obrazu
image_path = "img/apple1.jpg"

# Załaduj model YOLO
yolo_model = YOLO('foduucom/plant-leaf-detection-and-classification')
yolo_model.overrides['conf'] = 0.25  # Ustawienia detekcji
yolo_model.overrides['iou'] = 0.45
yolo_model.overrides['agnostic_nms'] = False
yolo_model.overrides['max_det'] = 1000

# Detekcja
results = yolo_model.predict(image_path)
boxes = results[0].boxes.data.numpy()  # [x_min, y_min, x_max, y_max, conf, class]
image = Image.open(image_path)

# Wyświetlenie wyniku detekcji
render = render_result(model=yolo_model, image=image, result=results[0])
render.show()

# Zapisanie wykrytych obszarów
detected_boxes = []
for box in boxes:
    x_min, y_min, x_max, y_max, conf, cls = box
    detected_boxes.append((x_min, y_min, x_max, y_max, int(cls)))


Ultralytics YOLOv8.0.43  Python-3.11.10 torch-2.5.1+cpu CPU
Model summary (fused): 168 layers, 11143386 parameters, 0 gradients, 28.5 GFLOPs

image 1/1 C:\Users\midon\Desktop\STUDIA WSB\_SEM 3\2. Multimedia\projekt\yolo_ds\img\apple1.jpg: 640x640 1 apple, 140.1ms
Speed: 1.0ms preprocess, 140.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


3. Preprocessing obrazów do klasyfikacji

In [14]:
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np

# Funkcja do preprocessingu obrazów dla Twojego modelu
def preprocess_image(cropped_image):
    resized_image = cropped_image.resize((224, 224))  # Rozmiar wejściowy zgodny z modelem
    array_image = img_to_array(resized_image) / 255.0  # Normalizacja pikseli
    input_image = np.expand_dims(array_image, axis=0)  # Dodanie wymiaru batch
    return input_image

# Przygotowanie obrazów do klasyfikacji
preprocessed_images = []
for box in detected_boxes:
    x_min, y_min, x_max, y_max, cls = map(int, box)  # Zamiana na liczby całkowite
    cropped_image = image.crop((x_min, y_min, x_max, y_max))  # Przycięcie obrazu
    input_image = preprocess_image(cropped_image)  # Przygotowanie obrazu
    preprocessed_images.append((input_image, cls))  # Obraz i klasa YOLO


4. Klasyfikacja wykrytych obszarów

In [15]:
# Klasyfikacja wykrytych obszarów
pathogen_predictions = []
for input_image, yolo_class in preprocessed_images:
    prediction = classifier_model.predict(input_image)  # Klasyfikacja modelem
    predicted_class = np.argmax(prediction, axis=1)[0]  # Klasa z najwyższym prawdopodobieństwem
    confidence = np.max(prediction)  # Pewność predykcji
    pathogen_predictions.append((yolo_class, predicted_class, confidence))  # Klasa YOLO, patogen, pewność

    # Wyświetlenie wyniku
    print(f"YOLO detected class: {yolo_class}, Pathogen class: {predicted_class}, Confidence: {confidence:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
YOLO detected class: 28, Pathogen class: 2, Confidence: 1.00


5. Mapowanie klas

In [16]:
# Lista nazw roślin
plants_list = [
    'ginger', 'banana', 'tobacco', 'ornamaental', 'rose', 'soyabean', 'papaya',
    'garlic', 'raspberry', 'mango', 'cotton', 'corn', 'pomgernate', 'strawberry',
    'Blueberry', 'brinjal', 'potato', 'wheat', 'olive', 'rice', 'lemon', 'cabbage',
    'gauava', 'chilli', 'capcicum', 'sunflower', 'cherry', 'cassava', 'apple', 'tea',
    'sugarcane', 'groundnut', 'weed', 'peach', 'coffee', 'cauliflower', 'tomato',
    'onion', 'gram', 'chiku', 'jamun', 'castor', 'pea', 'cucumber', 'grape', 'cardamom'
]

# Przekształcenie listy na słownik z indeksami
plants_dict = {index: plant for index, plant in enumerate(plants_list)}

pathogen_classes = {
    0: "Bacterial Disease",
    1: "Fungal Infection",
    2: "Healthy Plant",
    3: "Viral Disease"
}

6. Generowanie promptów

In [17]:
# Generowanie promptu na podstawie wyników
for yolo_class, predicted_class, confidence in pathogen_predictions:
    yolo_class_name = plants_dict.get(yolo_class, "Unknown Plant")
    pathogen_class_name = pathogen_classes.get(predicted_class, "Unknown Pathogen")

    if pathogen_class_name != "Healthy Plant":  # Jeśli wykryto chorobę
        prompt = (
            f"A {pathogen_class_name} has been detected on a {yolo_class_name} "
            f"with a confidence of {confidence:.2f}. "
            f"How can I prevent or treat this disease?"
        )
    else:  # Jeśli roślina jest zdrowa
        prompt = (
            f"The detected {yolo_class_name} is healthy with a confidence of {confidence:.2f}. "
            f"Do you have any suggestions for maintaining the health of this plant?"
        )

    # Wyświetlenie promptu
    print("Generated Prompt:")
    print(prompt)


Generated Prompt:
The detected apple is healthy with a confidence of 1.00. Do you have any suggestions for maintaining the health of this plant?


7. Integracja GPT z Openai API:

In [18]:
import os
from openai import OpenAI

# Ścieżka do pliku z kluczem API
key_file_path = os.path.join(".env", "OPENAI_API_KEY.txt")

# Odczyt klucza API z pliku
with open(key_file_path, "r") as key_file:
    APIKey = key_file.read().strip()

# Sprawdzenie, czy klucz został poprawnie załadowany
if not APIKey:
    raise ValueError("Plik z kluczem API jest pusty lub niepoprawny.")

# Inicjalizacja OpenAI
client = OpenAI(
    api_key=APIKey  # Klucz API wczytany z pliku
)

In [19]:
system_prompt = "Jestem ekspertem ds. roślin. Odpowiem na pytania dotyczące chorób i pielęgnacji."

In [20]:
response = client.chat.completions.create(
    model="gpt-4o-mini",  # Model GPT-4o-mini
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7
    )

# Wyświetlenie odpowiedzi
print("GPT Response:")
print(response.choices[0].message.content.strip())
print("Response Usage:", dict(response).get('usage'))

01/09/2025 22:04:19 - INFO - httpx -   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 401 Unauthorized"


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************LU0A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}